In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker
import numpy as np

print("--- Iniciando a etapa de Transformação de Dados ---")

# Define o caminho da pasta de entrada (onde os CSVs limpos estão)
# Considerando que este novo notebook estará na pasta 'notebooks/'
input_dir = os.path.join('..', 'data', 'output')

# Carregue os DataFrames limpos
df_products_clean = pd.read_csv(os.path.join(input_dir, 'products_cleaned.csv'))
df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))
df_salesgoals_clean = pd.read_csv(os.path.join(input_dir, 'salesgoals_cleaned.csv'))
df_salespeople_clean = pd.read_csv(os.path.join(input_dir, 'salespeople_cleaned.csv'))
df_commercialprojects_clean = pd.read_csv(os.path.join(input_dir, 'commercialprojects_cleaned.csv'))
df_mediamaps_clean = pd.read_csv(os.path.join(input_dir, 'mediamaps_cleaned.csv'))

print("DataFrames limpos carregados com sucesso no notebook de Transformação.")

# --- CONVERSÃO DE TIPOS DE DADOS PARA DATETIME NOVAMENTE ---
# Isso é crucial, pois ao carregar de CSV, as datas são lidas como strings
print("\nConvertendo colunas de data para o tipo datetime...")

df_products_clean['data_cadastro'] = pd.to_datetime(df_products_clean['data_cadastro'])
df_sales_clean['data_venda'] = pd.to_datetime(df_sales_clean['data_venda'])
# 'ano_meta' em salesgoals_clean pode precisar de um formato específico se não for apenas o ano.
# Assumindo que é apenas o ano, podemos tratar como string para merge e depois converter se necessário.
# Por enquanto, se 'ano_meta' for '2020' por exemplo, pd.to_datetime('2020', format='%Y') funcionaria.
# Se for '2020-01-01', pd.to_datetime já deve inferir. Vamos checar o tipo primeiro.
# df_salesgoals_clean['ano_meta'] = pd.to_datetime(df_salesgoals_clean['ano_meta'], format='%Y') # Descomente/ajuste se necessário.

df_commercialprojects_clean['data_inicio'] = pd.to_datetime(df_commercialprojects_clean['data_inicio'])
df_commercialprojects_clean['data_fim'] = pd.to_datetime(df_commercialprojects_clean['data_fim'])

print("Colunas de data convertidas para datetime.")

print("\nAgora você pode começar suas transformações!")


--- Iniciando a etapa de Transformação de Dados ---
DataFrames limpos carregados com sucesso no notebook de Transformação.

Convertendo colunas de data para o tipo datetime...
Colunas de data convertidas para datetime.

Agora você pode começar suas transformações!


In [2]:
print("--- Colunas de Cada DataFrame ---")

# Lista de DataFrames para iterar (assumindo que já estão carregados)
dataframes_to_inspect = [
    ('df_products_clean', df_products_clean),
    ('df_sales_clean', df_sales_clean),
    ('df_salesgoals_clean', df_salesgoals_clean),
    ('df_salespeople_clean', df_salespeople_clean),
    ('df_commercialprojects_clean', df_commercialprojects_clean),
    ('df_mediamaps_clean', df_mediamaps_clean)
]

for df_name, df_object in dataframes_to_inspect:
    print(f"\nDataFrame: {df_name}")
    print("Colunas:", df_object.columns.tolist())

print("\n--- Fim da Listagem ---")

--- Colunas de Cada DataFrame ---

DataFrame: df_products_clean
Colunas: ['id', 'nome_produto', 'custo_producao_brl', 'margem_desejada_percentual', 'status', 'data_cadastro']

DataFrame: df_sales_clean
Colunas: ['id', 'projeto_id', 'data_venda', 'valor_final_brl', 'quantidade_vendida', 'vendedor_id', 'regiao', 'cota_meta_vendas_id']

DataFrame: df_salesgoals_clean
Colunas: ['id', 'ano', 'mes', 'meta_vendas_brl']

DataFrame: df_salespeople_clean
Colunas: ['id', 'nome_vendedor', 'email', 'telefone']

DataFrame: df_commercialprojects_clean
Colunas: ['id', 'nome_projeto', 'produto_id', 'data_inicio', 'data_fim', 'valor_proposto_brl', 'status_negociacao']

DataFrame: df_mediamaps_clean
Colunas: ['id', 'projeto_id', 'canal_midia', 'data_veiculacao', 'custo_midia_brl', 'observacoes']

--- Fim da Listagem ---


In [14]:
print("--- Gerando Tabela Simples: ID do Vendedor, Total de Vendas, Nome e REGIÃO (ATUALIZADO) ---")

# --- 0. Carregar DataFrames Necessários ---
input_dir = os.path.join('..', 'data', 'output')

df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))
df_salespeople_clean = pd.read_csv(os.path.join(input_dir, 'salespeople_cleaned.csv'))

print("DataFrames 'df_sales_clean' e 'df_salespeople_clean' carregados.")

# --- 1. Calcular o Total de Vendas por Vendedor ID E REGIÃO ---
# AGORA AGRUPAMOS POR 'vendedor_id' E 'regiao'
total_vendas_por_vendedor_e_regiao = df_sales_clean.groupby(['vendedor_id', 'regiao']).agg(
    total_vendas_brl=('valor_final_brl', "sum")
).reset_index()

# --- 2. Mesclar com o Nome do Vendedor ---
# Mesclamos o resultado do agrupamento com os nomes dos vendedores
tabela_final_top_vendedores = pd.merge(
    total_vendas_por_vendedor_e_regiao,
    df_salespeople_clean[['id', 'nome_vendedor']],
    left_on='vendedor_id',
    right_on='id',
    how='left'
)

# --- 3. Limpeza Final e Reordenamento ---
# Remover a coluna 'id' duplicada que veio do merge
tabela_final_top_vendedores.drop(columns='id', inplace=True)

# Preencher nomes de vendedores que podem não ter sido encontrados
tabela_final_top_vendedores['nome_vendedor'] = tabela_final_top_vendedores['nome_vendedor'].fillna('Vendedor Desconhecido')

# Reordenar as colunas para a ordem desejada e ordenar por vendas
tabela_final_top_vendedores = tabela_final_top_vendedores[[
    'vendedor_id',
    'nome_vendedor', # Nome do vendedor vem antes da região para agrupar melhor visualmente
    'regiao',        # NOVA COLUNA AQUI!
    'total_vendas_brl'
]].sort_values(by='total_vendas_brl', ascending=False)


print("\n--- Tabela Final: Vendedor, Nome, Região e Total de Vendas (ATUALIZADO) ---")
print("Esta tabela inclui agora a região de cada vendedor para cada segmento de venda.")
print(tabela_final_top_vendedores.head(10))

print("\nDataFrame 'tabela_final_top_vendedores' criado e ATUALIZADO com sucesso!")

# --- Definição e Criação da Pasta de Destino ---
processed_tables_dir = os.path.join(input_dir, 'processed_tables')

if not os.path.exists(processed_tables_dir):
    os.makedirs(processed_tables_dir)
    print(f"Pasta '{processed_tables_dir}' criada.")
else:
    print(f"Pasta '{processed_tables_dir}' já existe.")

# --- Salvar a Tabela Atualizada de Top Vendedores ---
# O nome do arquivo pode continuar o mesmo, pois estamos sobrescrevendo com a versão melhorada
output_file_path = os.path.join(processed_tables_dir, 'vendedores_top_vendas.csv')
tabela_final_top_vendedores.to_csv(output_file_path, index=False)

print(f"DataFrame 'tabela_final_top_vendedores' ATUALIZADO e salvo com sucesso em: '{output_file_path}'")
print("\n--- Processo Concluído ---")

--- Gerando Tabela Simples: ID do Vendedor, Total de Vendas, Nome e REGIÃO (ATUALIZADO) ---
DataFrames 'df_sales_clean' e 'df_salespeople_clean' carregados.

--- Tabela Final: Vendedor, Nome, Região e Total de Vendas (ATUALIZADO) ---
Esta tabela inclui agora a região de cada vendedor para cada segmento de venda.
    vendedor_id    nome_vendedor        regiao  total_vendas_brl
32          107   Heloísa Britto         Norte       73253828.88
37          108    Igor Carvalho         Norte       73029606.80
0           101       Bruna Lima  Centro-Oeste       72763533.31
40          109   Janaína Dantas  Centro-Oeste       72621779.37
45          110   Kleber Esteves  Centro-Oeste       72528766.65
10          103     Denise Ramos  Centro-Oeste       72264303.48
18          104    Elias Barbosa       Sudeste       72150643.19
34          107   Heloísa Britto           Sul       71742063.77
3           101       Bruna Lima       Sudeste       71671585.74
29          106  Guilherme Alves    

In [10]:
print("--- Gerando Tabela: Total de Vendas por Região ---")

# --- 0. Carregar DataFrame Necessário ---
# Define o caminho base para a pasta 'output'
input_dir = os.path.join('..', 'data', 'output') # Assumindo que o script está em 'notebooks/'

# Carrega o DataFrame de vendas limpas
df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))

print("DataFrame 'df_sales_clean' carregado.")

# --- 1. Calcular o Total de Vendas por Região ---
# Agrupamos por 'regiao' e somamos 'valor_final_brl'
total_vendas_por_regiao = df_sales_clean.groupby('regiao').agg(
    total_vendas_brl=('valor_final_brl', "sum") # Usando "sum" para evitar o FutureWarning
).reset_index()

# Opcional: Ordenar pela maior venda para facilitar a visualização
total_vendas_por_regiao = total_vendas_por_regiao.sort_values(by='total_vendas_brl', ascending=False)

print("\n--- Tabela Final: Total de Vendas por Região ---")
print("Esta tabela mostra o desempenho de vendas por cada região.")
print(total_vendas_por_regiao.head()) # Exibindo as primeiras linhas

print("\nDataFrame 'total_vendas_por_regiao' criado com sucesso!")

# --- 2. Salvar a Tabela Processada na Pasta 'processed_tables' ---

# O caminho para a pasta processed_tables
processed_tables_dir = os.path.join(input_dir, 'processed_tables')

# Cria a pasta se ela não existir (boa prática de segurança)
if not os.path.exists(processed_tables_dir):
    os.makedirs(processed_tables_dir)
    print(f"Pasta '{processed_tables_dir}' criada.")
else:
    print(f"Pasta '{processed_tables_dir}' já existe.")

# Salvar o DataFrame como CSV na pasta
output_file_path = os.path.join(processed_tables_dir, 'total_vendas_por_regiao.csv')
total_vendas_por_regiao.to_csv(output_file_path, index=False)

print(f"DataFrame 'total_vendas_por_regiao' salvo com sucesso em: '{output_file_path}'")
print("\n--- Processo Concluído ---")

--- Gerando Tabela: Total de Vendas por Região ---
DataFrame 'df_sales_clean' carregado.

--- Tabela Final: Total de Vendas por Região ---
Esta tabela mostra o desempenho de vendas por cada região.
         regiao  total_vendas_brl
0  Centro-Oeste      6.949129e+08
3       Sudeste      6.881054e+08
2         Norte      6.874352e+08
4           Sul      6.824541e+08
1      Nordeste      6.694622e+08

DataFrame 'total_vendas_por_regiao' criado com sucesso!
Pasta '..\data\output\processed_tables' já existe.
DataFrame 'total_vendas_por_regiao' salvo com sucesso em: '..\data\output\processed_tables\total_vendas_por_regiao.csv'

--- Processo Concluído ---


In [12]:
print("--- Gerando Tabela: Desempenho de Vendas por Produto (Versão Corrigida) ---")

# --- 0. Carregar DataFrames Necessários ---
input_dir = os.path.join('..', 'data', 'output')

df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))
df_products_clean = pd.read_csv(os.path.join(input_dir, 'products_cleaned.csv'))
df_commercialprojects_clean = pd.read_csv(os.path.join(input_dir, 'commercialprojects_cleaned.csv')) # AGORA INCLUÍDO!

print("DataFrames 'df_sales_clean', 'df_products_clean' e 'df_commercialprojects_clean' carregados.")

# --- 1. Preparar Dados: Juntar Vendas com Projetos e depois com Produtos ---

# Primeiro merge: df_sales_clean (venda) com df_commercialprojects_clean (projeto)
# Isso trará o 'produto_id' de dentro do projeto para a linha da venda.
vendas_com_projeto = pd.merge(
    df_sales_clean,
    df_commercialprojects_clean[['id', 'produto_id']], # Seleciona apenas id do projeto e o produto_id associado
    left_on='projeto_id', # Coluna de df_sales_clean que referencia o projeto
    right_on='id',        # Coluna de df_commercialprojects_clean que é o ID do projeto
    how='left',
    suffixes=('_venda', '_projeto')
)

# Renomear a coluna 'produto_id' vinda do merge do projeto para evitar conflito e ser clara
vendas_com_projeto.rename(columns={'produto_id': 'id_produto_associado'}, inplace=True)
# Remover a coluna 'id_projeto' duplicada (era 'id' de df_commercialprojects_clean)
vendas_com_projeto.drop(columns='id_projeto', inplace=True, errors='ignore') # Usamos errors='ignore' para segurança


# Segundo merge: o resultado (vendas_com_projeto) com df_products_clean (detalhes do produto)
# Agora usamos 'id_produto_associado' para ligar à tabela de produtos.
vendas_com_detalhes_produto = pd.merge(
    vendas_com_projeto,
    df_products_clean[['id', 'nome_produto', 'custo_producao_brl', 'margem_desejada_percentual']],
    left_on='id_produto_associado', # Usa o ID do produto que veio do projeto
    right_on='id',                  # ID do produto em df_products_clean
    how='left',
    suffixes=('_transacao', '_produto') # Ajustado sufixos para clareza
)

# Limpeza de colunas duplicadas e renomeações para clareza no DataFrame final
vendas_com_detalhes_produto.drop(columns=['id_transacao', 'id_produto'], inplace=True, errors='ignore') # Remover IDs duplicados
vendas_com_detalhes_produto.rename(columns={'id_transacao_venda': 'id_venda'}, inplace=True) # ID original da venda


# --- 2. Calcular Lucro por Transação (antes de agrupar) ---
if 'valor_final_brl' in vendas_com_detalhes_produto.columns and \
   'custo_producao_brl' in vendas_com_detalhes_produto.columns and \
   'quantidade_vendida' in vendas_com_detalhes_produto.columns:

    vendas_com_detalhes_produto['lucro_brl_por_transacao'] = \
        vendas_com_detalhes_produto['valor_final_brl'] - \
        (vendas_com_detalhes_produto['custo_producao_brl'] * vendas_com_detalhes_produto['quantidade_vendida'])
    print("Coluna 'lucro_brl_por_transacao' calculada.")
else:
    print("Colunas necessárias para cálculo de lucro por transação não encontradas. Lucro não será calculado.")
    vendas_com_detalhes_produto['lucro_brl_por_transacao'] = 0


# --- 3. Agrupar por Item (Produto) ---
desempenho_por_produto = vendas_com_detalhes_produto.groupby('nome_produto').agg(
    total_quantidade_vendida=('quantidade_vendida', "sum"),
    total_valor_vendas_brl=('valor_final_brl', "sum"),
    total_lucro_brl=('lucro_brl_por_transacao', "sum")
).reset_index()

# Calcular a margem de lucro percentual do produto
desempenho_por_produto['margem_lucro_percentual'] = \
    np.where(desempenho_por_produto['total_valor_vendas_brl'] > 0,
             (desempenho_por_produto['total_lucro_brl'] / desempenho_por_produto['total_valor_vendas_brl']) * 100,
             0)

desempenho_por_produto = desempenho_por_produto.sort_values(by='total_valor_vendas_brl', ascending=False)


print("\n--- Tabela Final: Desempenho de Vendas por Produto (Gerada) ---")
print(desempenho_por_produto.head())
print("\nDataFrame 'desempenho_por_produto' criado com sucesso!")

# --- 4. Salvar a Tabela Processada na Pasta 'processed_tables' ---

processed_tables_dir = os.path.join(input_dir, 'processed_tables')

if not os.path.exists(processed_tables_dir):
    os.makedirs(processed_tables_dir)
    print(f"Pasta '{processed_tables_dir}' criada.")
else:
    print(f"Pasta '{processed_tables_dir}' já existe.")

output_file_path = os.path.join(processed_tables_dir, 'desempenho_por_produto.csv')
desempenho_por_produto.to_csv(output_file_path, index=False)

print(f"DataFrame 'desempenho_por_produto' salvo com sucesso em: '{output_file_path}'")
print("\n--- Processo Concluído ---")

--- Gerando Tabela: Desempenho de Vendas por Produto (Versão Corrigida) ---
DataFrames 'df_sales_clean', 'df_products_clean' e 'df_commercialprojects_clean' carregados.
Coluna 'lucro_brl_por_transacao' calculada.

--- Tabela Final: Desempenho de Vendas por Produto (Gerada) ---
                         nome_produto  total_quantidade_vendida  \
98                Ventilador de Torre                      5630   
97  Vaso Sanitário com Caixa Acoplada                      5331   
80                   Roteador Wi-Fi 6                     13868   
75        Pistola de Pintura Elétrica                      6303   
45                     GPS Automotivo                     12615   

    total_valor_vendas_brl  total_lucro_brl  margem_lucro_percentual  
98             60974613.09      33732506.89                55.322216  
97             55259270.76      28990981.50                52.463562  
80             53408335.62      29719850.10                55.646464  
75             53205558.85      264

In [13]:
print("--- Gerando Tabela: Desempenho de Vendas por Item e Região ---")

# --- 0. Carregar DataFrames Necessários ---
input_dir = os.path.join('..', 'data', 'output')

df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))
df_products_clean = pd.read_csv(os.path.join(input_dir, 'products_cleaned.csv'))
df_commercialprojects_clean = pd.read_csv(os.path.join(input_dir, 'commercialprojects_cleaned.csv'))

print("DataFrames 'df_sales_clean', 'df_products_clean' e 'df_commercialprojects_clean' carregados.")

# --- 1. Preparar Dados: Juntar Vendas com Projetos e depois com Produtos ---

# Primeiro merge: df_sales_clean (venda) com df_commercialprojects_clean (projeto)
# Objetivo: Trazer o 'produto_id' para cada transação de venda.
vendas_com_projeto = pd.merge(
    df_sales_clean,
    df_commercialprojects_clean[['id', 'produto_id']], # Seleciona id do projeto e o produto_id associado
    left_on='projeto_id', # Coluna de df_sales_clean que referencia o projeto
    right_on='id',        # Coluna de df_commercialprojects_clean que é o ID do projeto
    how='left',
    suffixes=('_venda', '_projeto')
)

# Renomear a coluna 'produto_id' vinda do merge do projeto para ser a chave de merge com produtos
vendas_com_projeto.rename(columns={'produto_id': 'id_produto_associado'}, inplace=True)
# Remover a coluna 'id' duplicada que veio de df_commercialprojects_clean
vendas_com_projeto.drop(columns='id_projeto', inplace=True, errors='ignore')


# Segundo merge: o resultado (vendas_com_projeto) com df_products_clean (detalhes do produto)
# Objetivo: Trazer o 'nome_produto' e outros detalhes do produto.
vendas_com_detalhes_produto_e_regiao = pd.merge(
    vendas_com_projeto,
    df_products_clean[['id', 'nome_produto']], # Precisamos do id e nome do produto
    left_on='id_produto_associado', # Usa o ID do produto que veio do projeto
    right_on='id',                  # ID do produto em df_products_clean
    how='left',
    suffixes=('_transacao', '_produto')
)

# Limpeza de colunas duplicadas e renomeações
vendas_com_detalhes_produto_e_regiao.drop(columns=['id_produto'], inplace=True, errors='ignore') # Remover IDs duplicados
vendas_com_detalhes_produto_e_regiao.rename(columns={'id_transacao_venda': 'id_venda'}, inplace=True)


# --- 2. Agrupar por Região e Nome do Produto ---
# Agregamos a quantidade e o valor total de vendas por 'regiao' e 'nome_produto'
vendas_por_item_e_regiao = vendas_com_detalhes_produto_e_regiao.groupby(['regiao', 'nome_produto']).agg(
    total_quantidade_vendida=('quantidade_vendida', "sum"),
    total_valor_vendas_brl=('valor_final_brl', "sum")
).reset_index()

# Opcional: Ordenar para ver os itens de maior faturamento por região
vendas_por_item_e_regiao = vendas_por_item_e_regiao.sort_values(
    by=['regiao', 'total_valor_vendas_brl'],
    ascending=[True, False] # Ordena por região (A-Z) e depois por valor (maior para menor)
)


print("\n--- Tabela Final: Desempenho de Vendas por Item e Região ---")
print("Esta tabela mostra a quantidade e o valor total para cada item em cada região.")
print(vendas_por_item_e_regiao.head(10)) # Exibindo as primeiras 10 linhas

print("\nDataFrame 'vendas_por_item_e_regiao' criado com sucesso!")

# --- 3. Salvar a Tabela Processada na Pasta 'processed_tables' ---

processed_tables_dir = os.path.join(input_dir, 'processed_tables')

# Cria a pasta se ela não existir (boa prática de segurança)
if not os.path.exists(processed_tables_dir):
    os.makedirs(processed_tables_dir)
    print(f"Pasta '{processed_tables_dir}' criada.")
else:
    print(f"Pasta '{processed_tables_dir}' já existe.")

# Salvar o DataFrame como CSV na pasta
output_file_path = os.path.join(processed_tables_dir, 'vendas_por_item_e_regiao.csv')
vendas_por_item_e_regiao.to_csv(output_file_path, index=False)

print(f"DataFrame 'vendas_por_item_e_regiao' salvo com sucesso em: '{output_file_path}'")
print("\n--- Processo Concluído ---")

--- Gerando Tabela: Desempenho de Vendas por Item e Região ---
DataFrames 'df_sales_clean', 'df_products_clean' e 'df_commercialprojects_clean' carregados.

--- Tabela Final: Desempenho de Vendas por Item e Região ---
Esta tabela mostra a quantidade e o valor total para cada item em cada região.
          regiao                       nome_produto  total_quantidade_vendida  \
98  Centro-Oeste                Ventilador de Torre                      1413   
97  Centro-Oeste  Vaso Sanitário com Caixa Acoplada                      1331   
80  Centro-Oeste                   Roteador Wi-Fi 6                      3009   
0   Centro-Oeste          Abajur de Chão Decorativo                      1278   
65  Centro-Oeste          Mesa de Jantar 4 Cadeiras                      3936   
86  Centro-Oeste            Serra Elétrica Circular                      2065   
57  Centro-Oeste                Lanterna Tática LED                      4728   
75  Centro-Oeste        Pistola de Pintura Elétrica    

In [20]:
# --- BLOCO DE CÓDIGO ATUALIZADO NO SEU ARQUIVO DE TRANSFORMAÇÃO DE DADOS ---

import pandas as pd
import os

print("\n--- Iniciando Geração da Tabela: Vendas por Produto, Vendedor e Região ---")

# --- 0. Carregar DataFrames Necessários ---
input_dir = os.path.join('..', 'data', 'output')

print("Carregando DataFrames de origem...")
df_sales_clean = pd.read_csv(os.path.join(input_dir, 'sales_cleaned.csv'))
df_products_clean = pd.read_csv(os.path.join(input_dir, 'products_cleaned.csv'))
df_salespeople_clean = pd.read_csv(os.path.join(input_dir, 'salespeople_cleaned.csv'))
df_commercialprojects_clean = pd.read_csv(os.path.join(input_dir, 'commercialprojects_cleaned.csv'))
print("DataFrames de origem carregados com sucesso!")


# --- 1. Mesclar os DataFrames para obter todas as informações necessárias ---

# Passo 1: Juntar df_sales_clean com df_commercialprojects_clean para obter o produto_id
print("1/3: Mesclando df_sales_clean com df_commercialprojects_clean para obter 'produto_id'...")
# Selecionar apenas as colunas necessárias ANTES do merge para evitar IDs desnecessários
df_proj_info = df_commercialprojects_clean[['id', 'produto_id']].copy()
df_proj_info.rename(columns={'id': 'projeto_id_origem_proj'}, inplace=True) # Renomeia 'id' para evitar conflito futuro

df_vendas_detalhada = pd.merge(df_sales_clean,
                               df_proj_info,
                               left_on='projeto_id',
                               right_on='projeto_id_origem_proj',
                               how='left')
df_vendas_detalhada.drop(columns='projeto_id_origem_proj', inplace=True) # Remove a coluna de junção auxiliar


# Passo 2: Juntar o resultado com df_products_clean para obter o nome_produto
print("2/3: Mesclando com df_products_clean para obter 'nome_produto'...")
# Selecionar apenas as colunas necessárias ANTES do merge
df_prod_info = df_products_clean[['id', 'nome_produto']].copy()
df_prod_info.rename(columns={'id': 'produto_id_origem_prod'}, inplace=True) # Renomeia 'id' para evitar conflito futuro

df_vendas_detalhada = pd.merge(df_vendas_detalhada,
                               df_prod_info,
                               left_on='produto_id',
                               right_on='produto_id_origem_prod',
                               how='left')
df_vendas_detalhada.drop(columns='produto_id_origem_prod', inplace=True) # Remove a coluna de junção auxiliar


# Passo 3: Juntar o resultado com df_salespeople_clean para obter o nome_vendedor
print("3/3: Mesclando com df_salespeople_clean para obter 'nome_vendedor'...")
# Selecionar apenas as colunas necessárias ANTES do merge
df_salesperson_info = df_salespeople_clean[['id', 'nome_vendedor']].copy()
df_salesperson_info.rename(columns={'id': 'vendedor_id_origem_sp'}, inplace=True) # Renomeia 'id' para evitar conflito futuro

df_vendas_detalhada = pd.merge(df_vendas_detalhada,
                               df_salesperson_info,
                               left_on='vendedor_id',
                               right_on='vendedor_id_origem_sp',
                               how='left')
df_vendas_detalhada.drop(columns='vendedor_id_origem_sp', inplace=True) # Remove a coluna de junção auxiliar


# Tratar possíveis valores ausentes para nomes
df_vendas_detalhada['nome_produto'] = df_vendas_detalhada['nome_produto'].fillna('Produto Desconhecido')
df_vendas_detalhada['nome_vendedor'] = df_vendas_detalhada['nome_vendedor'].fillna('Vendedor Desconhecido')


# --- 2. Agrupar para obter o total de vendas por Região, Vendedor e Produto ---
df_vendas_produto_vendedor_regiao = df_vendas_detalhada.groupby([
    'regiao',
    'nome_vendedor',
    'nome_produto'
]).agg(
    total_vendas_brl=('valor_final_brl', 'sum'),
    quantidade_vendida=('quantidade_vendida', 'sum')
).reset_index()

# Opcional: Ordenar para facilitar a inspeção inicial, por região e depois por vendas
df_vendas_produto_vendedor_regiao = df_vendas_produto_vendedor_regiao.sort_values(
    by=['regiao', 'total_vendas_brl'], ascending=[True, False]
)

print("\n--- Primeiras linhas da nova tabela 'vendas_produto_vendedor_regiao' ---")
print(df_vendas_produto_vendedor_regiao.head())
print(f"Total de linhas na nova tabela: {len(df_vendas_produto_vendedor_regiao)}")


# --- 3. Salvar a nova tabela processada ---
processed_tables_dir = os.path.join(input_dir, 'processed_tables')
os.makedirs(processed_tables_dir, exist_ok=True)

output_file_path = os.path.join(processed_tables_dir, 'vendas_produto_vendedor_regiao.csv')
df_vendas_produto_vendedor_regiao.to_csv(output_file_path, index=False)

print(f"DataFrame 'vendas_produto_vendedor_regiao.csv' salvo com sucesso em: '{output_file_path}'")
print("\n--- Geração da tabela 'vendas_produto_vendedor_regiao.csv' concluída ---")


--- Iniciando Geração da Tabela: Vendas por Produto, Vendedor e Região ---
Carregando DataFrames de origem...
DataFrames de origem carregados com sucesso!
1/3: Mesclando df_sales_clean com df_commercialprojects_clean para obter 'produto_id'...
2/3: Mesclando com df_products_clean para obter 'nome_produto'...
3/3: Mesclando com df_salespeople_clean para obter 'nome_vendedor'...

--- Primeiras linhas da nova tabela 'vendas_produto_vendedor_regiao' ---
           regiao   nome_vendedor                       nome_produto  \
627  Centro-Oeste  Heloísa Britto                  Chaleira Elétrica   
197  Centro-Oeste     Caio Mendes  Vaso Sanitário com Caixa Acoplada   
698  Centro-Oeste  Heloísa Britto                Ventilador de Torre   
80   Centro-Oeste      Bruna Lima                   Roteador Wi-Fi 6   
65   Centro-Oeste      Bruna Lima          Mesa de Jantar 4 Cadeiras   

     total_vendas_brl  quantidade_vendida  
627        2413009.85                 360  
197        2269132.48   